In [60]:
from utils.kmers import kMerDF, kMer
import numpy as np
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import stats
from Bio import SeqIO

In [32]:
#Load attention scores, convert to numpy array
expl_scores = pd.read_csv('examples/DNABERTexpl.csv')['scores']
expl_scores = [str(i).split(' ') for i in expl_scores]
expl_scores = np.array(expl_scores).astype(float)

validation_labels = np.array(pd.read_csv('examples/DNABERTexpl.csv')['label'])

#load DNA sequences and their labels, convert to numpy arrays.
dnadf = pd.read_csv('examples/XLTestingData.csv')['b3']
dnadf = np.array([str(i).split(' ') for i in dnadf])
labels = pd.read_csv('examples/XLTestingData.csv')['label'].to_numpy()


#make sure all the sequences in both dataframes are sorted in the same order
assert validation_labels.all() == labels.all()

In [61]:
#create the kMerDF object
X = kMerDF(expl_scores, dnadf, labels)

#get the kMer object at row 0, column 0
print('kMerDF[0][506]: ' + X.get_kmer(0, 506).seq)

#showing this is the same kmer as the first in the dnadf
print('dnadf[0][506]: ' + dnadf[0][506])

AttributeError: 'kMerDF' object has no attribute 'kmerdf'

In [53]:
len(X.filter_kmers(1.0))

47803